In [1]:
incompatibilidades = []
tiempos_lavado = {}
cantidad_prendas = 0
cantidad_incompatibilidades = 0

with open('datos.txt', 'r') as file:
    for line in file:
        if line.startswith('p'):
            _, _,cantidad_prendas, cantidad_incompatibilidades = line.split()
        if line.startswith('e'):
            _, prenda1, prenda2 = line.split()
            incompatibilidades.append((int(prenda1), int(prenda2)))
        if line.startswith('n'):
            _, numero_prenda, tiempo_lavado = line.split()
            tiempos_lavado[int(numero_prenda)] = int(tiempo_lavado)

cantidad_prendas = int(cantidad_prendas)
cantidad_incompatibilidades = int(cantidad_incompatibilidades)

print("Incompatibilidades: ", incompatibilidades)
print("Tiempos lavado: ", tiempos_lavado)

print("Chequeo que tengan el largo correcto: ")
print(f"Incompatibilidades: {cantidad_incompatibilidades == len(incompatibilidades)}")
print(f"Tiempos lavado: {cantidad_prendas == len(tiempos_lavado)}")



Incompatibilidades:  [(1, 10), (1, 19), (1, 2), (1, 3), (1, 4), (1, 5), (1, 6), (1, 7), (1, 8), (1, 9), (1, 17), (2, 11), (2, 20), (2, 9), (2, 3), (2, 4), (2, 5), (2, 6), (2, 7), (2, 8), (2, 10), (2, 18), (2, 1), (3, 12), (3, 10), (3, 17), (3, 4), (3, 5), (3, 6), (3, 7), (3, 8), (3, 11), (3, 19), (3, 2), (3, 1), (4, 13), (4, 11), (4, 18), (4, 5), (4, 6), (4, 7), (4, 8), (4, 12), (4, 20), (4, 3), (4, 2), (4, 1), (5, 14), (5, 12), (5, 19), (5, 6), (5, 7), (5, 8), (5, 13), (5, 4), (5, 3), (5, 2), (5, 1), (6, 15), (6, 13), (6, 20), (6, 7), (6, 8), (6, 14), (6, 5), (6, 4), (6, 3), (6, 2), (6, 1), (7, 16), (7, 14), (7, 8), (7, 15), (7, 6), (7, 5), (7, 4), (7, 3), (7, 2), (7, 1), (8, 15), (8, 16), (8, 7), (8, 6), (8, 5), (8, 4), (8, 3), (8, 2), (8, 1), (9, 18), (9, 10), (9, 11), (9, 12), (9, 13), (9, 14), (9, 15), (9, 16), (9, 17), (9, 2), (9, 1), (10, 19), (10, 17), (10, 11), (10, 12), (10, 13), (10, 14), (10, 15), (10, 16), (10, 18), (10, 9), (10, 3), (10, 2), (10, 1), (11, 20), (11, 18), (

In [2]:
from pulp import *

# Crear una instancia del problema de minimización
problema = LpProblem("Problema de Lavandería", LpMinimize)

# Definir las variables de decisión
prendas = list(tiempos_lavado.keys())
lavados = range(1, len(prendas) + 1)

x = LpVariable.dicts("Lavo prenda i en lavado j", [(i, j) for i in prendas for j in prendas], cat="Binary")
y = LpVariable.dicts("Lavado i, siendo i la prenda que mas demora", [(i) for i in lavados], cat="Binary")

# Definir la función objetivo
problema += lpSum(tiempos_lavado[i] * y[i] for i in prendas)

# Definir las restricciones

# Cada prenda debe lavarse exactamente una vez
for i in prendas:
    problema += lpSum(x[(i, j)] for j in prendas) == 1

for i in prendas:
    for j in prendas:
        if i != j:
            problema += x[(i, j)] + x[(j, i)] <= 1

# No se pueden lavar prendas incompatibles juntas
for (i, j) in incompatibilidades:
    problema += x[(i, j)] == 0
    problema += x[(j, i)] == 0

# Relaciono prendas que se levan con su lavado
for j in prendas:
    problema += lpSum(x[(i, j)] for i in prendas) >= y[j]
    problema += lpSum(x[(i, j)] for i in prendas) <= y[j] * cantidad_prendas


# Resolver el problema
problema.solve()

# Imprimir el estado de la solución
print("Estado:", LpStatus[problema.status])

print(problema)

Estado: Optimal
Problema_de_Lavandería:
MINIMIZE
8*Lavado_i,_siendo_i_la_prenda_que_mas_demora_1 + 8*Lavado_i,_siendo_i_la_prenda_que_mas_demora_10 + 6*Lavado_i,_siendo_i_la_prenda_que_mas_demora_11 + 9*Lavado_i,_siendo_i_la_prenda_que_mas_demora_12 + 5*Lavado_i,_siendo_i_la_prenda_que_mas_demora_13 + 1*Lavado_i,_siendo_i_la_prenda_que_mas_demora_14 + 5*Lavado_i,_siendo_i_la_prenda_que_mas_demora_15 + 9*Lavado_i,_siendo_i_la_prenda_que_mas_demora_16 + 10*Lavado_i,_siendo_i_la_prenda_que_mas_demora_17 + 4*Lavado_i,_siendo_i_la_prenda_que_mas_demora_18 + 4*Lavado_i,_siendo_i_la_prenda_que_mas_demora_19 + 10*Lavado_i,_siendo_i_la_prenda_que_mas_demora_2 + 5*Lavado_i,_siendo_i_la_prenda_que_mas_demora_20 + 9*Lavado_i,_siendo_i_la_prenda_que_mas_demora_3 + 7*Lavado_i,_siendo_i_la_prenda_que_mas_demora_4 + 3*Lavado_i,_siendo_i_la_prenda_que_mas_demora_5 + 9*Lavado_i,_siendo_i_la_prenda_que_mas_demora_6 + 10*Lavado_i,_siendo_i_la_prenda_que_mas_demora_7 + 2*Lavado_i,_siendo_i_la_prenda_que_ma

c:\Users\Pc\AppData\Local\Programs\Python\Python310\lib\site-packages\pulp\pulp.py:1316: UserWarning: Spaces are not permitted in the name. Converted to '_'
  warnings.warn("Spaces are not permitted in the name. Converted to '_'")


In [13]:
for i in lavados:
    print(f"Lavado {i} funcionando: {value(y[i])}")

for i in prendas:
    for j in prendas:
        if (value(x[i,j]) == 1.0):
            print(f"Prenda {i} lavado {j}: {value(x[i,j])}")



Lavado 1 funcionando: 0.0
Lavado 2 funcionando: 0.0
Lavado 3 funcionando: 0.0
Lavado 4 funcionando: 0.0
Lavado 5 funcionando: 0.0
Lavado 6 funcionando: 0.0
Lavado 7 funcionando: 0.0
Lavado 8 funcionando: 1.0
Lavado 9 funcionando: 0.0
Lavado 10 funcionando: 0.0
Lavado 11 funcionando: 0.0
Lavado 12 funcionando: 0.0
Lavado 13 funcionando: 0.0
Lavado 14 funcionando: 1.0
Lavado 15 funcionando: 0.0
Lavado 16 funcionando: 0.0
Lavado 17 funcionando: 0.0
Lavado 18 funcionando: 1.0
Lavado 19 funcionando: 0.0
Lavado 20 funcionando: 0.0
Prenda 1 lavado 18: 1.0
Prenda 2 lavado 14: 1.0
Prenda 3 lavado 18: 1.0
Prenda 4 lavado 14: 1.0
Prenda 5 lavado 18: 1.0
Prenda 6 lavado 18: 1.0
Prenda 7 lavado 18: 1.0
Prenda 8 lavado 14: 1.0
Prenda 9 lavado 8: 1.0
Prenda 10 lavado 8: 1.0
Prenda 11 lavado 8: 1.0
Prenda 12 lavado 8: 1.0
Prenda 13 lavado 8: 1.0
Prenda 14 lavado 18: 1.0
Prenda 15 lavado 18: 1.0
Prenda 16 lavado 18: 1.0
Prenda 17 lavado 14: 1.0
Prenda 18 lavado 18: 1.0
Prenda 19 lavado 8: 1.0
Prenda 20